# 6. Symbolics using SymPy

## Setup

In [1]:
using Pkg # hide
Pkg.activate("/home/travis/build/JuliaRobotics/RigidBodyDynamics.jl/docs/../examples/6. Symbolics using SymPy") # hide
Pkg.instantiate() # hide
using RigidBodyDynamics
using StaticArrays
using SymPy

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Installed VersionParsing ─ v1.1.3
 Installed Conda ────────── v1.2.0
 Installed PyCall ───────── v1.91.2
 Installed SymPy ────────── v1.0.3
  Building Conda ─→ `~/.julia/packages/Conda/CpuvI/deps/build.log`
  Building PyCall → `~/.julia/packages/PyCall/ttONZ/deps/build.log`
┌ Info: Installing sympy via the Conda sympy package...
└ @ PyCall /home/travis/.julia/packages/PyCall/ttONZ/src/PyCall.jl:705
┌ Info: Running `conda install -q -y sympy` in root environment
└ @ Conda /home/travis/.julia/packages/Conda/CpuvI/src/Conda.jl:112
Solving environment: ...working... done

## Package Plan ##

  environment location: /home/travis/.julia/conda/3

  added / updated specs:
    - sympy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    fastcache-1.1.0            |   py37h7b6447

## Create symbolic parameters
* Masses: $m_1, m_2$
* Mass moments of inertia (about center of mass): $I_1, I_2$
* Link lengths: $l_1, l_2$
* Center of mass locations (w.r.t. preceding joint axis): $c_1, c_2$
* Gravitational acceleration: $g$

In [2]:
inertias = @syms m_1 m_2 I_1 I_2 positive = true
lengths = @syms l_1 l_2 c_1 c_2 real = true
gravitational_acceleration = @syms g real = true
params = [inertias..., lengths..., gravitational_acceleration...]
transpose(params)

1×9 LinearAlgebra.Transpose{SymPy.Sym,Array{SymPy.Sym,1}}:
 m_1  m_2  I_1  I_2  l_1  l_2  c_1  c_2  g

## Create double pendulum `Mechanism`
A `Mechanism` contains the joint layout and inertia parameters, but no state information.

In [3]:
T = Sym # the 'scalar type' of the Mechanism we'll construct
axis = SVector(zero(T), one(T), zero(T)) # axis of rotation for each of the joints
double_pendulum = Mechanism(RigidBody{T}("world"); gravity = SVector(zero(T), zero(T), g))
world = root_body(double_pendulum) # the fixed 'world' rigid body

RigidBody: "world"

Attach the first (upper) link to the world via a revolute joint named 'shoulder'

In [4]:
inertia1 = SpatialInertia(CartesianFrame3D("upper_link"), moment=I_1 * axis * transpose(axis), com=SVector(zero(T), zero(T), c_1), mass=m_1)
body1 = RigidBody(inertia1)
joint1 = Joint("shoulder", Revolute(axis))
joint1_to_world = one(Transform3D{T}, frame_before(joint1), default_frame(world));
attach!(double_pendulum, world, body1, joint1, joint_pose = joint1_to_world);

Attach the second (lower) link to the world via a revolute joint named 'elbow'

In [5]:
inertia2 = SpatialInertia(CartesianFrame3D("lower_link"), moment=I_2 * axis * transpose(axis), com=SVector(zero(T), zero(T), c_2), mass=m_2)
body2 = RigidBody(inertia2)
joint2 = Joint("elbow", Revolute(axis))
joint2_to_body1 = Transform3D(frame_before(joint2), default_frame(body1), SVector(zero(T), zero(T), l_1))
attach!(double_pendulum, body1, body2, joint2, joint_pose = joint2_to_body1)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

## Create `MechanismState` associated with the double pendulum `Mechanism`
A `MechanismState` stores all state-dependent information associated with a `Mechanism`.

In [6]:
x = MechanismState(double_pendulum);

Set the joint configuration vector of the MechanismState to a new vector of symbolic variables

In [7]:
q = configuration(x)
for i in eachindex(q)
    q[i] = symbols("q_$i", real = true)
end

Set the joint velocity vector of the MechanismState to a new vector of symbolic variables

In [8]:
v = velocity(x)
for i in eachindex(v)
    v[i] = symbols("v_$i", real = true)
end

## Compute dynamical quantities in symbolic form

Mass matrix

In [9]:
simplify.(mass_matrix(x))

2×2 Array{SymPy.Sym,2}:
 I_1 + I_2 + 2*c_2*l_1*m_2*cos(q_2) + l_1^2*m_2  I_2 + c_2*l_1*m_2*cos(q_2)
                     I_2 + c_2*l_1*m_2*cos(q_2)                         I_2

Kinetic energy

In [10]:
simplify(kinetic_energy(x))

     2        2                   2                                           
I₁⋅v₁    I₂⋅v₁               I₂⋅v₂               2                            
────── + ────── + I₂⋅v₁⋅v₂ + ────── + c₂⋅l₁⋅m₂⋅v₁ ⋅cos(q₂) + c₂⋅l₁⋅m₂⋅v₁⋅v₂⋅co
  2        2                   2                                              

          2      2
        l₁ ⋅m₂⋅v₁ 
s(q₂) + ──────────
            2     

Potential energy

In [11]:
simplify(gravitational_potential_energy(x))

-g⋅(c₁⋅m₁⋅cos(q₁) + c₂⋅m₂⋅cos(q₁ + q₂) + l₁⋅m₂⋅cos(q₁))

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*